# CNN Video Sentiment Analysis
This notebook will walk through the process of training a CNN to analyse frames of videos, and then take the aggregate over frames in a video.

In [1]:
import matplotlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import random
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os


In [2]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: charlieisalright (use `wandb login --relogin` to force relogin)


True

In [3]:
# Setup the location of the various parameters needed

dataset = "../data/img_dataset"
epochs = 25

labels = set(["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"])

We use transfer learning, on top of the ResNet CNN, using frames extracted from our videos to get the specific model. In this case, we are training using data at 720p.

In [4]:
image_paths = list(paths.list_images(dataset))
random.seed(10)
random.shuffle(image_paths)
image_paths = image_paths[:3000]
data = []
labels = []

print(len(image_paths))

for image_path in image_paths:
    label = image_path.split(os.path.sep)[-2]
    
    # Load images, converting to RGB channels and scaling to 224 x 224
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    
    data.append(image)
    labels.append(label)
    
print("Done fetching images")
    
    
data = np.array(data)
labels = np.array(labels)

# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels, 
                        test_size=0.25, stratify=labels, random_state=42)

# Randomly change the train set so results are more generalizable
train_augmentation = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

val_augmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_augmentation.mean = mean
val_augmentation.mean = mean

3000
Done fetching images


In [5]:
def get_model():
    base_model = ResNet50(weights="imagenet", include_top=False, 
                      input_tensor=Input(shape=(224, 224, 3)))

    head_model = base_model.output
    head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
    head_model = Flatten(name="flatten")(head_model)
    head_model = Dense(512, activation="relu")(head_model)
    head_model = Dropout(0.5)(head_model)
    head_model = Dense(len(lb.classes_), activation="softmax")(head_model)

    model = Model(inputs=base_model.input, outputs=head_model)

    # Ensure we don't train the base model
    for layer in base_model.layers:
        layer.trainable = False
        
    return model


In [6]:
def train():
    # default hyperparameters
    config_defaults = {
        'batch_size' : 32,
        'learning_rate' : 0.001,
        'epochs': 25
    }

    wandb.init(project='sentiment', entity='charlieisalright', config=config_defaults)
    config = wandb.config
    
    config.architecture_name = "ResNet50"
    
    # Compile the model, using stochastic gradient descent optimization.
    opt = SGD(lr=config.learning_rate, momentum=0.9, decay=1e-4 / config.epochs)
    model = get_model()
    model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

    # Now we can start training!
    H = model.fit(
        x = train_augmentation.flow(trainX, trainY, batch_size=config.batch_size),
        steps_per_epoch = len(trainX) // 32,
        validation_data = val_augmentation.flow(testX, testY),
        validation_steps = len(testX) // 32,
        epochs = config.epochs,
        callbacks = [WandbCallback()]
    )


Next, we setup a sweep of hyperparameters.

In [ ]:
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val_loss",
        "goal": "minimize"
    },
    "parameters":{
        "epochs": {
            "distribution": "int_uniform",
            "min": 13,
            "max": 50
        },
        "batch_size": {
            "distribution": "int_uniform",
            "min": 4,
            "max": 64
        },
        "learning_rate": {
            "distribution": "uniform",
            "min": 0.00001,
            "max": 0.01
        }
    }
}
sweep_id = "fnf3wbru"
wandb.agent(sweep_id, project='sentiment', function=train)

wandb: Agent Starting Run: zd82erpg with config:
wandb: 	batch_size: 59
wandb: 	epochs: 41
wandb: 	learning_rate: 0.006203575112534187
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep
/home/charlie/Documents/courses/miniproject/env/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


Epoch 1/41
39/70 [===============>..............] - 43s 1s/step - loss: 2.2777 - accuracy: 0.1538 - val_loss: 2.0229 - val_accuracy: 0.1970


epoch,0
loss,2.27771
accuracy,0.15378
val_loss,2.02285
val_accuracy,0.19701
_runtime,58
_timestamp,1617047955
_step,0
best_val_loss,2.02285
best_epoch,0


epoch,▁
loss,▁
accuracy,▁
val_loss,▁
val_accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: Agent Starting Run: gqrp7e89 with config:
wandb: 	batch_size: 12
wandb: 	epochs: 46
wandb: 	learning_rate: 0.003954295645280029
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


...

wandb: Agent Starting Run: nwlrev61 with config:
wandb: 	batch_size: 13
wandb: 	epochs: 50
wandb: 	learning_rate: 0.0011110784180112267
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Epoch 1/50
70/70 [==============================] - 22s 310ms/step - loss: 2.4345 - accuracy: 0.1374 - val_loss: 2.0176 - val_accuracy: 0.1875
Epoch 2/50
70/70 [==============================] - 19s 278ms/step - loss: 2.0713 - accuracy: 0.1737 - val_loss: 2.0394 - val_accuracy: 0.1766
Epoch 3/50
70/70 [==============================] - 20s 286ms/step - loss: 2.0354 - accuracy: 0.1989 - val_loss: 1.9949 - val_accuracy: 0.2364
Epoch 4/50
70/70 [==============================] - 20s 281ms/step - loss: 2.0290 - accuracy: 0.2033 - val_loss: 2.0070 - val_accuracy: 0.2337
Epoch 5/50
70/70 [==============================] - 20s 286ms/step - loss: 2.0176 - accuracy: 0.1989 - val_loss: 1.9684 - val_accuracy: 0.2351
Epoch 6/50
70/70 [==============================] - 20s 285ms/step - loss: 1.9986 - accuracy: 0.1989 - val_loss: 1.9671 - val_accuracy: 0.2147
Epoch 7/50
70/70 [==============================] - 21s 293ms/step - loss: 2.0032 - accuracy: 0.2165 - val_loss: 1.9615 - val_accuracy: 0.2024

epoch,49
loss,1.81315
accuracy,0.31737
val_loss,1.75225
val_accuracy,0.33016
_runtime,1035
_timestamp,1617091011
_step,49
best_val_loss,1.72165
best_epoch,46


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁
accuracy,▁▂▃▃▃▄▄▄▄▅▅▅▅▆▅▆▅▆▆▆▅▅▆▇▆▇▆▇▆▆▆▇▇▇▇▇██▇█
val_loss,██▇▇▆▆▆▆▆▅▅▅▄▄▄▄▄▅▃▄▄▄▃▄▃▃▂▄▃▃▂▂▂▂▂▂▂▁▂▂
val_accuracy,▁▁▃▃▂▂▃▄▂▅▅▄▅▄▅▇▇▅▆▅▅▄▆▄▆▇▇▄▄▇▇▆▅▆▆▆▆█▆▆
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: 5b6r98nr with config:
wandb: 	batch_size: 61
wandb: 	epochs: 13
wandb: 	learning_rate: 0.00027551091944849046
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


Finally, evaluate the network, and plot some results.

In [ ]:
predictions = model.predict(x=testX.astype("float32"), batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

N = epochs
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()

Finally, serialize model and label binarizer to file.

In [ ]:
model.save("model/activity.model", save_formt="h5")
with f = open("model/lb.pickle", "wb"):
    f.write(pickle.dumps(lb))
    

## NOTES
This works but provides accuracy of only about 0.3 ish. Next thing to try is using a face details model first!!!